In [1]:
pip install apyori

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=0707044078ee5cc9ded9c17ddf8a546d37de000928f1e6bcb03cb8db634deb22
  Stored in directory: c:\users\khali\appdata\local\pip\cache\wheels\32\2a\54\10c595515f385f3726642b10c60bf788029e8f3a1323e3913a
Successfully built apyori
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install ipython --upgrade

     -------------------------------------- 808.2/808.2 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 386.4/386.4 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.20
    Uninstalling prompt-toolkit-3.0.20:
      Successfully uninstalled prompt-toolkit-3.0.20
  Attempting uninstall: ipython
    Found existing installation: ipython 7.31.1
    Uninstalling ipython-7.31.1:
      Successfully uninstalled ipython-7.31.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
spyder 5.2.2 requires ipython<8.0.0,>=7.6.0, but you have ipython 8.18.1 which is incompatible.
spyder-kernels 2.2.1 requires ipython<8,>=7.6.0; python_version >= "3", but you have ipython 8.18.1 which is incompatible.


In [7]:
import pandas as pd
from apyori import apriori

In [16]:
import pandas as pd
from apyori import apriori

# Step 1: Load the CSV file
df = pd.read_csv('transactions.csv')

# Step 2: Prepare the transactions
# Split the "Items" column by commas to get the list of items for each transaction
transactions = df['Items'].apply(lambda x: x.split(',')).tolist()

# Step 3: Apply Apriori algorithm
association_rules = apriori(transactions, min_support=0.1, min_confidence=0.3)

# Convert the results into a list for easier manipulation
rules_list = list(association_rules)

# Initialize a dictionary to store frequent itemsets of any size
frequent_itemsets = {}

In [17]:
# Step 4: Extract frequent itemsets and rules
formatted_rules = []
for rule in rules_list:
    itemset = list(rule.items)
    itemset_length = len(itemset)

    # If this is the first time encountering this itemset length, initialize an empty list for it
    if itemset_length not in frequent_itemsets:
        frequent_itemsets[itemset_length] = []

    # Store the itemsets in the appropriate list based on their length
    frequent_itemsets[itemset_length].append((itemset, rule.support))

    for result in rule.ordered_statistics:
        antecedent = list(result.items_base)
        consequent = list(result.items_add)

        if antecedent and consequent:
            support = rule.support
            confidence = result.confidence
            lift = result.lift

            # Separate antecedent and consequent into different columns, and count their lengths
            formatted_rules.append({
                'Antecedent (If)': ', '.join(antecedent),
                'Consequent (Then)': ', '.join(consequent),
                'Support': round(support, 4),
                'Confidence': round(confidence, 4),
                'Lift': round(lift, 4),
                'Antecedent Length': len(antecedent),  # Number of items in the antecedent
                'Consequent Length': len(consequent),  # Number of items in the consequent
            })

# Step 5: Create DataFrames for frequent itemsets
itemsets_df_list = {}
for length, itemsets in frequent_itemsets.items():
    if itemsets:
        itemsets_df = pd.DataFrame(itemsets, columns=[f'{length}-Itemsets', 'Support'])
        itemsets_df_list[length] = itemsets_df

# Step 6: Create a DataFrame for the rules
rules_df = pd.DataFrame(formatted_rules)

# Step 7: Display frequent itemsets and association rules
for length, df in itemsets_df_list.items():
    print(f"\nFrequent {length}-Itemsets:")
    print(df)

print("\nAssociation Rules:")
print(rules_df)

# Step 8: Optionally, save the results to CSV files
for length, df in itemsets_df_list.items():
    df.to_csv(f"frequent_{length}_itemsets.csv", index=False)

# Save rules with antecedent and consequent in separate columns, and include lengths
rules_df.to_csv("association_rules_with_lengths.csv", index=False)


Frequent 1-Itemsets:
      1-Itemsets  Support
0      [ apples]   0.3685
1       [ bread]   0.3570
2      [ butter]   0.3785
3      [ cereal]   0.3345
4      [ cheese]   0.3550
5   [ chocolate]   0.3670
6        [ eggs]   0.3710
7         [ jam]   0.3580
8        [ milk]   0.3645
9     [ oranges]   0.3770
10        [ tea]   0.3455

Frequent 2-Itemsets:
                2-Itemsets  Support
0        [ apples,  bread]   0.1475
1       [ butter,  apples]   0.1535
2       [ apples,  cereal]   0.1370
3       [ apples,  cheese]   0.1470
4    [ apples,  chocolate]   0.1590
5         [ eggs,  apples]   0.1430
6          [ apples,  jam]   0.1425
7         [ apples,  milk]   0.1480
8      [ oranges,  apples]   0.1445
9          [ apples,  tea]   0.1315
10       [ butter,  bread]   0.1460
11       [ cereal,  bread]   0.1270
12       [ bread,  cheese]   0.1360
13    [ chocolate,  bread]   0.1490
14         [ eggs,  bread]   0.1450
15          [ jam,  bread]   0.1435
16         [ bread,  milk]   0.1